In [7]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

In [8]:
df_housing = pd.read_csv('housing_data.csv')
df_housing.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [9]:
# Add interaction variables
df_housing['DIS_LSTAT'] = df_housing['DIS'] * df_housing['LSTAT']
df_housing['ZN_RM']  = df_housing['ZN'] * df_housing['RM']
df_housing['RAD_LSTAT'] = df_housing['RAD'] * df_housing['LSTAT']
df_housing['RAD_TAX']  = df_housing['RAD'] * df_housing['TAX'] 
df_housing['CRIM_ZN']  = df_housing['CRIM'] * df_housing['ZN'] 

In [10]:
df_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CRIM       506 non-null    float64
 1   ZN         506 non-null    float64
 2   INDUS      506 non-null    float64
 3   CHAS       506 non-null    int64  
 4   NOX        506 non-null    float64
 5   RM         506 non-null    float64
 6   AGE        506 non-null    float64
 7   DIS        506 non-null    float64
 8   RAD        506 non-null    int64  
 9   TAX        506 non-null    int64  
 10  PTRATIO    506 non-null    float64
 11  B          506 non-null    float64
 12  LSTAT      506 non-null    float64
 13  PRICE      506 non-null    float64
 14  DIS_LSTAT  506 non-null    float64
 15  ZN_RM      506 non-null    float64
 16  RAD_LSTAT  506 non-null    float64
 17  RAD_TAX    506 non-null    int64  
 18  CRIM_ZN    506 non-null    float64
dtypes: float64(15), int64(4)
memory usage: 75.2 KB


In [11]:
# To scale numeric features, select numeric features
df_housing_numeric = df_housing.drop(['PRICE','CHAS'], axis=1).select_dtypes(include = np.number)
df_housing_numeric.head()

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,DIS_LSTAT,ZN_RM,RAD_LSTAT,RAD_TAX,CRIM_ZN
0,0.00632,18.0,2.31,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,20.368200,118.35,4.98,296,0.11376
1,0.02731,0.0,7.07,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,45.399294,0.00,18.28,484,0.00000
2,0.02729,0.0,7.07,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,20.017413,0.00,8.06,484,0.00000
3,0.03237,0.0,2.18,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,17.822868,0.00,8.82,666,0.00000
4,0.06905,0.0,2.18,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,32.311526,0.00,15.99,666,0.00000


In [12]:
# Scale the numeric features and 
X_scaler = StandardScaler()
num_scaled=X_scaler.fit_transform(df_housing_numeric)
df_num_scaled = pd.DataFrame(num_scaled,columns = df_housing_numeric.columns)
df_num_scaled.head()

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,DIS_LSTAT,ZN_RM,RAD_LSTAT,RAD_TAX,CRIM_ZN
0,-0.419782,0.284830,-1.287909,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562,-0.887781,0.262418,-0.744390,-0.769714,-0.355332
1,-0.417339,-0.487722,-0.593381,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439,0.212752,-0.480021,-0.676661,-0.740395,-0.405081
2,-0.417342,-0.487722,-0.593381,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727,-0.903204,-0.480021,-0.728706,-0.740395,-0.405081
3,-0.416750,-0.487722,-1.306878,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517,-0.999690,-0.480021,-0.724835,-0.712011,-0.405081
4,-0.412482,-0.487722,-1.306878,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501,-0.362673,-0.480021,-0.688322,-0.712011,-0.405081


In [13]:
X = pd.concat([df_num_scaled, df_housing['CHAS']], axis=1)
y = df_housing['PRICE']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
print('Train R2 ', train_r2)
print('Test R2  ', test_r2)

Train R2  0.7747867689198509
Test R2   0.7152181788736488


# Regularization

## Ridge

In [20]:
# Build and evaluate Ridge Model

ridge_model = Ridge(alpha=0.2)                  # alpha is lambda
ridge_model.fit(X_train,y_train)

y_pred_train = ridge_model.predict(X_train)
y_pred_test = ridge_model.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
print('Train R2 ', train_r2)
print('Test R2  ', test_r2)

Train R2  0.7744500869035429
Test R2   0.7160489066296205


In [18]:
print(lr.coef_)

[ -0.66739885 -13.20799295  -0.09951027  -1.87721423   2.06884408
   0.10478023  -1.1479755    4.32857691  -2.13794424  -1.7416942
   0.79686113  -1.64894603  -1.1664528   13.78824469  -1.65452558
  -0.89448501   0.54796729   2.56751236]


In [19]:
print(ridge_model.coef_)

[ -0.66754801 -11.28245751  -0.11964369  -1.88276135   2.1515178
   0.08165754  -1.2066585    3.91732326  -2.19539345  -1.77559396
   0.80750218  -1.57957077  -1.20043823  11.86182797  -1.66780057
  -0.41938327   0.52296145   2.57065402]


## Lasso

In [24]:
# Build and evaluate Lasso Model

lasso_model = Lasso(alpha=0.1)                  # alpha is lambda
lasso_model.fit(X_train,y_train)

y_pred_train = lasso_model.predict(X_train)
y_pred_test = lasso_model.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
print('Train R2 ', train_r2)
print('Test R2  ', test_r2)

Train R2  0.7504245026458002
Test R2   0.6905841315257106


In [25]:
print(lasso_model.coef_)

[-0.52310958 -0.         -0.03844633 -1.32441029  2.73971427 -0.
 -1.27115015  1.15793659 -1.35879331 -1.77403131  0.86080792 -2.31612743
 -1.00990604  0.47187462 -0.          0.          0.29719969  1.66474172]


## Elastic Net

In [27]:
# Build and evaluate ElasticNet Model

en_model = ElasticNet(alpha=1, l1_ratio=0.8)                  # alpha is lambda
en_model.fit(X_train,y_train)

y_pred_train = en_model.predict(X_train)
y_pred_test = en_model.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
print('Train R2 ', train_r2)
print('Test R2  ', test_r2)

Train R2  0.6854366091770645
Test R2   0.6054900378993648


In [28]:
print(en_model.coef_)

[-0.          0.         -0.28695691 -0.1449559   2.37725877 -0.
  0.         -0.         -0.38563084 -1.15799779  0.38547639 -2.09780143
 -0.81360718  0.02740275 -0.31914314 -0.          0.11160619  0.        ]


## Grid Search

In [29]:
from sklearn.model_selection import GridSearchCV

In [31]:
# Grid Search - Ridge
param = {'alpha':[0.01, 0.1, 1, 10]}

model = Ridge()

grid_cv = GridSearchCV(estimator=model, param_grid=param, cv=5, scoring='r2')       # cv=5(default), scoring='r2'(default)
grid_cv.fit(X_train, y_train)
grid_cv.best_params_

{'alpha': 0.1}

In [36]:
# Lasso
param = {'alpha':[0.01, 0.1, 1, 10]}

model = Lasso(max_iter=2000) # Required if non-convergence warning is recieved

grid_cv = GridSearchCV(estimator=model, param_grid=param, cv=5, scoring='r2')       # cv=5(default), scoring='r2'(default)
grid_cv.fit(X_train, y_train)
grid_cv.best_params_

{'alpha': 0.01}

In [40]:
# Elastic Net
param = {'alpha':[0.01, 0.1, 0.3, 0.5],
        'l1_ratio':[0.3, 0.5, 0.7, 0.9]}


algo_name = ElasticNet()

grid_cv = GridSearchCV(estimator=algo_name, param_grid=param, cv=5, scoring='r2')       # cv=5(default), scoring='r2'(default)
grid_cv.fit(X_train, y_train)
grid_cv.best_params_

{'alpha': 0.01, 'l1_ratio': 0.9}